# Sample Form filled with hand written text

- Extract using Azure Document Intelligence Service
- Draw Insights and detect fraud with AOAI GPT-4

## Prerequisites
1. To run the code, install the following packages. Please use the latest pre-release version `pip install azure-ai-formrecognizer==3.3.0`.


- > ! pip install azure-ai-formrecognizer==3.3.0

## Load all the API keys, parameters and login credentials

In [1]:
import fr

# Your Azure Document Intelligence Service Instance
MY_FORM_RECOGNIZER_ENDPOINT = 'https://tr-docai-form-recognizer.cognitiveservices.azure.com/'
# The model id should match the custom model you have
# trained and deployed in your Azure Document Intelligence Service Instance
# with the endpoint MY_FORM_RECOGNIZER_ENDPOINT
MY_CLAIMS_MODEL_ID = 'claims-v3'

formRecognizerCredential = fr.getFormRecognizerCredential()

formRecognizerClient = fr.getDocumentAnalysisClient(
                            endpoint=MY_FORM_RECOGNIZER_ENDPOINT,
                            credential=formRecognizerCredential
                        )


Got Azure Form Recognizer API Key from environment variable


## Document Extraction Examples

### Auto Insurance Claims form by hand

- Custom Trained model
- Display label, data and confidence (document level and indivudual field level)
- Text, Checkbox, radio button

#### Display labeled data

In [20]:
# Assuming you are running notebook from the notebook folder
#MY_TEST_DOCUMENT = r'..\..\..\data\sample-claims-docs\testing\IC-handwritten-RobertFrost.pdf'
MY_TEST_DOCUMENT = r'..\..\..\data\sample-claims-docs\testing\IC-handwritten-JeanGenet.pdf'

fr_api_version, model_id, is_handwritten, result = fr.extractResultFromLocalDocument(
                                                        client=formRecognizerClient,
                                                        model=MY_CLAIMS_MODEL_ID,
                                                        filepath=MY_TEST_DOCUMENT
                                                    )

print(f'Document Intelligence API version = {fr_api_version}\n \
        Document Extraction Model Id = {model_id}\n \
        Does document have any hand written text? {is_handwritten}\n'
     )
doc_count = len(result.documents)
print(f'Document count = {doc_count}')

IncidentDescription = 'None'
SustainerInjuries = 'None'
IncidentLocation = 'None'
IncidentTime = 'None'
IncidentDate = 'None'

for idx, document in enumerate(result.documents):
    print(f'Document {idx} ---------------')
    print(f'\tDocument extraction confidence = {document.confidence}')
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print("\t{}[type:{};conf:{}] = '{}'".format(name, field.value_type, field.confidence, field_value))
        if name == 'IncidentDate':
            IncidentDate = field_value
        if name == 'IncidentTime':
            IncidentTime = field_value
        if name == 'SustainedInjuries':
            SustainedInjuries = field_value
        if name == 'IncidentDescription':
            IncidentDescription = field_value
        if name == 'IncidentLocation':
            IncidentLocation = field_value            

Document Intelligence API version = 2023-07-31
         Document Extraction Model Id = claims-v3
         Does document have any hand written text? True

Document count = 1
Document 0 ---------------
	Document extraction confidence = 0.987
	FormType[type:string;conf:0.938] = 'Auto Insurance Claim Document'
	Name[type:string;conf:0.965] = 'Jean Genet'
	Address[type:string;conf:0.908] = '12 Independence Way, Boston, MA 11201'
	Phone[type:string;conf:0.967] = '196 3412112'
	Email[type:string;conf:0.964] = 'jg@jean.com'
	PolicyNumber[type:string;conf:0.914] = 'TRI 123 320 918'
	IncidentDate[type:string;conf:0.96] = '1/1/2023'
	IncidentTime[type:string;conf:0.948] = '3am EST'
	IncidentLocation[type:string;conf:0.92] = '780 Lynnway, Lynn, MA 01905'
	IncidentDescription[type:string;conf:0.916] = 'Some one hit the back of the car parked at Walmart parking lot, while I was inside the store shopping.'
	VehicleOwner[type:string;conf:0.98] = 'Self'
	VehicleMakeAndModel[type:string;conf:0.95] = '20

## Post Processing after extraction, to draw insight, check for fraud

<b><u>Example 1</u></b>  
From <b>IncidentDescription</b> find if victim was inside the car at the time of the incident.  
And then from <b>SustainedInjuries</b> check if there was any personal injury reported.  

#### Load the AOAI keys and parameters

In [4]:
import aoai

MY_AOAI_ENDPOINT = 'https://tr-non-prod-gpt4.openai.azure.com/'
MY_AOAI_VERSION = '2023-07-01-preview'
MY_GPT_ENGINE = 'tr-gpt4'
MY_AOAI_EMBEDDING_ENGINE = 'tr-embedding-ada'

status = aoai.setupOpenai(aoai_endpoint=MY_AOAI_ENDPOINT, 
                 aoai_version=MY_AOAI_VERSION)
if status > 0:
    print("AOAI setup succeeded")
else:
    print("AOAI setup failed")


Got OPENAI API Key from environment variable
AOAI setup succeeded


#### Ask GPT-4 to draw insight

In [16]:
my_task = f'From {IncidentDescription} find if the person was inside the car. \
            If no, find from {SustainedInjuries} if there was any injury. \
            If the person was not in the car but sustained injury in this incident, is this not a fraud? \
            Answer if there was fraud : yes or no\
            Provide explanation'

my_prompt = [
              {
                "role": "user", 
                "content": my_task
                }
            ]      
tokens_used, finish_reason, aoai_answer = aoai.getChatCompletion(the_engine=MY_GPT_ENGINE, 
                                                                           the_messages=my_prompt)
print(f"Tokens: {tokens_used}")
print(f"Finish Reason: {finish_reason}")
print(f"Answer: {aoai_answer}")

Tokens: 214
Finish Reason: stop
Answer: The person was not inside the car during the incident as they were inside the store shopping. The person did sustain injuries such as a concussion with a head injury and a few bruises on their knees. However, these injuries could not have been caused by the car incident as the person was not in the car at the time. Therefore, this could be considered a fraud. 

Answer: Yes

Explanation: The person was not in the car when it was hit, so they could not have sustained injuries from this incident. Claiming injuries from an incident they were not involved in is fraudulent.


<b><u>Example 2</u></b>  
Find if there is a store at <b>IncidentLocation</b>   
Verify if that store is open at the <b>IncidentTime</b>  

In [35]:
# TODO: This part should be implemented with Bing Search API for better accuracy
print (f'Incident Data & Time: {IncidentDate} {IncidentTime}\nIncident location:{IncidentLocation}\n')

my_task = f'Name the store at {IncidentLocation}.\
            Answer:'

my_prompt = [
              {
                "role": "user", 
                "content": my_task
                }
            ]      
tokens_used, finish_reason, aoai_answer = aoai.getChatCompletion(the_engine=MY_GPT_ENGINE, 
                                                                           the_messages=my_prompt)
StoreName = aoai_answer
print(f"Tokens: {tokens_used}")
print(f"Finish Reason: {finish_reason}")
print(f"Answer: {StoreName}")

Incident Data & Time: 1/1/2023 3am EST
Incident location:780 Lynnway, Lynn, MA 01905

Tokens: 30
Finish Reason: stop
Answer: Walmart Supercenter


In [38]:
my_task = f'Is {StoreName} at {IncidentLocation} open at {IncidentTime}? \
            Answer yes or no\
            Explain:'

my_prompt = [
              {
                "role": "user", 
                "content": my_task
                }
            ]      
tokens_used, finish_reason, aoai_answer = aoai.getChatCompletion(the_engine=MY_GPT_ENGINE, 
                                                                           the_messages=my_prompt)
print(f"Tokens: {tokens_used}")
print(f"Finish Reason: {finish_reason}")
print(f"Answer: {aoai_answer}")

Tokens: 81
Finish Reason: stop
Answer: No

Explanation: Walmart Supercenter at 780 Lynnway, Lynn, MA 01905 is not open at 3am EST. Their typical hours of operation are from 7am to 11pm.


From <b>IncidentDescription</b> find what the victim was doing at the time of the incident

In [40]:
my_task = f'What was I doing based on the description in {IncidentDescription}? \
            Answer in one word'

my_prompt = [
              {
                "role": "user", 
                "content": my_task
                }
            ]      
tokens_used, finish_reason, aoai_answer = aoai.getChatCompletion(the_engine=MY_GPT_ENGINE, 
                                                                           the_messages=my_prompt)
print(f"Tokens: {tokens_used}")
print(f"Finish Reason: {finish_reason}")
print(f"Answer: {aoai_answer}")

Tokens: 44
Finish Reason: stop
Answer: Shopping
